In [1]:
import numpy as np
import pandas as pd
import googlemaps


In [105]:
areeVerdi = pd.read_csv('data/utf8_areeVerdiMilano.csv', sep=';')
queryAddresses = areeVerdi.iloc[:,3].values + ', Milano'

In [ ]:
gmaps = googlemaps.Client(key=key)

nLimit = 1500

# Geocoding an address
locationsFull = pd.Series.from_array([{}]*len(queryAddresses))
coordinates = pd.DataFrame([[np.nan, np.nan]]*len(queryAddresses), columns=['Long', 'Lat'])
multipleResults = pd.Series.from_array([]*len(queryAddresses))

for (i, address) in enumerate(queryAddresses[0:nLimit]): 
    geocode_result = gmaps.geocode(address, components={'location':'Milano'})
    if geocode_result:
        assert isinstance(geocode_result, list) #check type
        if len(geocode_result) > 1: # multiple results, report first and
            multipleResults[i] = geocode_result
        
        # take first output    
        locationsFull[i] = geocode_result[0]
            
        # parse coordinates
        thisLocation = locationsFull[i]['geometry']['location']
        coordinates['Long'][i] = thisLocation['lng']
        coordinates['Lat'][i] = thisLocation['lat']
        print(coordinates.iloc[i,:])
        
    print(i)


0
Long     9.184743
Lat     45.473632
Name: 1, dtype: float64
1
Long     9.184721
Lat     45.472865
Name: 2, dtype: float64
2
Long     9.193184
Lat     45.473830
Name: 3, dtype: float64
3
Long     9.192493
Lat     45.470208
Name: 4, dtype: float64
4
Long     9.192774
Lat     45.471542
Name: 5, dtype: float64
5
Long     9.189406
Lat     45.476821
Name: 6, dtype: float64
6
Long     9.178096
Lat     45.469185
Name: 7, dtype: float64
7
Long     9.181731
Lat     45.468658
Name: 8, dtype: float64
8
Long     9.181839
Lat     45.471492
Name: 9, dtype: float64
9
Long     9.182371
Lat     45.470140
Name: 10, dtype: float64
10
Long     9.181803
Lat     45.479775
Name: 11, dtype: float64
11
Long     9.179526
Lat     45.468033
Name: 12, dtype: float64
12
Long     9.180304
Lat     45.465379
Name: 13, dtype: float64
13


In [3]:
#coordinates.name = 'Localizzazione'
areeVerdiGeoloc = pd.concat([areeVerdi, coordinates], axis=1)

areeVerdiGeoloc.to_csv('utf8_Geoloc_areeVerdiMilano.csv', sep=';', decimal=',')
coordinates.to_csv('coordinateAreeVerdi.csv', sep=';', decimal=',')

NameError: name 'areeVerdi' is not defined

In [96]:
areeVerdiGeoloc

,Zona,Area,ID Localit�,Nome localit�,Descrizione,Tipo,Classificazione,Classificazione ISTAT,AFFIDATARIO,Superficie totale in mq,Long,Lat
0,1,1,1,piazza Castello - Minghetti,aiuola spartitraffico,Filare,Verde di arredo stradale,4 - Aree di arredo urbano,Appaltatore Servizio Manutenzione,"573,07",NaN,NaN
1,1,2,2,piazza Paolo VI,area verde,Parco,Giardino,1 - Verde attrezzato,Appaltatore Servizio Manutenzione,"1313,85",9.184743,45.473632
2,1,3,3,via San Simpliciano,filare alberato - parcheggio,Filare,Filare alberato,4 - Aree di arredo urbano,Appaltatore Servizio Manutenzione,"2,82",9.184721,45.472865
3,1,4,4,via De Marchi Marco,filare alberato,Filare,Filare alberato,4 - Aree di arredo urbano,Appaltatore Servizio Manutenzione,"212,28",9.193184,45.473830
4,1,5,5,via Croce Rossa,filare alberato,Filare,Filare alberato,4 - Aree di arredo urbano,Appaltatore Servizio Manutenzione,"6,47",9.192493,45.470208
5,1,6,6,via dei Giardini,filare alberato,Filare,Filare alberato,4 - Aree di arredo urbano,Appaltatore Servizio Manutenzione,"150,66",9.192774,45.471542
6,1,7,7,via Moscova,filare alberato,Filare,Filare alberato,4 - Aree di arredo urbano,Appaltatore Servizio Manutenzione,"104,42",9.189406,45.476821
7,1,8,8,piazza Castello,filare alberato,Filare,Filare alberato,4 - Aree di arredo urbano,Appaltatore Servizio Manutenzione,"1535,19",9.178096,45.469185
8,1,9,9,via Beltrami,filare alberato,Filare,Filare alberato,4 - Aree di arredo urbano,Appaltatore Servizio Manutenzione,"84,39",9.181731,45.468658
9,1,10,10,via Lanza,filare alberato,Filare,Filare alberato,4 - Aree di arredo urbano,Appaltatore Servizio Manutenzione,"28,65",9.181839,45.471492
